In [8]:
import glob
import sys
import cdms2 as cdms
import numpy as np
import MV2 as MV
import difflib
import scipy.stats as stats
global crunchy
import socket
import pickle
#import peakfinder as pf
import cdtime,cdutil,genutil
from eofs.cdms import Eof
from eofs.multivariate.cdms import MultivariateEof
import matplotlib.pyplot as plt
import matplotlib.cm as cm

sys.path.append("../python-utils")
import CMIP5_tools as cmip5
import DA_tools
import Plotting

### Set classic Netcdf (ver 3)
cdms.setNetcdfShuffleFlag(0)
cdms.setNetcdfDeflateFlag(0)
cdms.setNetcdfDeflateLevelFlag(0)
import os
import pandas as pd

In [9]:
import CMIP6Helper as cmip6

In [3]:
def penmont_vpd_SH(Tmax,Tmin,Rnet,RH,press,u):
    """
    
    Using the modified FAO Penman-Monteith approach, calculation reference
    (potential) evapotranspiration. As part of this, also calculate the vapor
    pressure deficit (VPD) and relative humidity RH.
    
    Inputs:
    Tmax    = maxiumum temperature, degrees K
    Tmin    = miniumum temperature, degrees K
    Rnet    = surface net radiation, W/m2
    u       = wind speed at 2 meters, m/s
    RH      = relative humidity, %
    press   = surface pressure, Pascals

    Outputs:
    PET  = potential evapotranspiration (mm/day)       
    VPD  = vapor presssure deficit (kPa)  
    RH   = relative humidity, fraction

    Written by Benjamin I. Cook and translated to Python by Kate Marvel
 
    Based on:
    Xu, C-Y., and V. P. Singh. "Cross comparison of empirical equations 
               for calculating potential evapotranspiration with data 
               from Switzerland." Water Resources Management,
               16.3 (2002): 197-219.

           FAO Document:
           http://www.fao.org/docrep/X0490E/x0490e00.htmContents

    For Tetens, both above and below zero:
       http://cires.colorado.edu/~voemel/vp.html
    """

    #Convert temperature to degrees C
    Tmax = Tmax -273.15
    Tmin = Tmin -273.15
    Ta=0.5+(Tmax+Tmin)
    
    # ground heat flux, set to zero (works fine on >monthly timescales, and is 
    # accurate if one calculates Rnet as SH+LH)
    gflux=0.

    #Extrapolate wind speed from 10m to 2m (ADDED BY KATE)
    u2 = u*(4.87/np.log(67.8*10.-5.42))


    # Calculate the latent heat of vaporization (MJ kg-1)
    lambda_lv=2.501-(2.361e-3)*Ta

    # Calculate Saturation vapor pressure (kPa) 
    es_max=0.611*np.exp((17.27*Tmax)/(Tmax+237.3))  
    es_min=0.611*np.exp((17.27*Tmin)/(Tmin+237.3))
    es=0.5*(es_max+es_min)

#     # Convert specific humidity (kg/kg) to actual vapor pressure
#     ea=(press*q)/0.6213 # Pascals

#     # Convert ea to kilopascals
#     ea = ea/1000.

    # Convert Pressure to kPa
    press=press/1000.

    # Use es and relative humidity to calculate ea in kPa
    ea=es.*RH/100.

    # Slope of the vapor pressure curve (kPa C-1)
    delta_vpc=(4098*es)/((Ta+237.3)**2)

    # Psychometric constant (kPa C-1)
    # 0.00163 is Cp (specific heat of moist air) divided by epsilon (0.622, 
    # ratio molecular weight of water vapour/dry air)
    psych_const=0.00163*(press/lambda_lv)

    # Net radiation
    Rnet=Rnet/11.6  # convert W/m2 to MJ/m2/d

    # Calculate VPD
    VPD=es-ea

#     # Calculate Relative Humidity
#     RH = ea/es

    # Potential Evapotranspiration (mm/day)
    PET=(0.408*delta_vpc*(Rnet-gflux)+psych_const*(900./(Ta+273))*u2*(VPD))/(delta_vpc+psych_const*(1+0.34*u))

    return PET, VPD#, RH


In [10]:
def PET_from_cmip(fname):
    """
    fname must be a tasmax file
    
    Calculate PET from CMIP6 data on dester
    #Tmax      = maxiumum temperature, degrees K = tasmax
    #Tmim      = miniumum temperature, degrees K = tasmin
    #   Rnet    = surface net radiation, W/m2 = (hfls+hfss)
    # u       = wind speed at the surface, m/s = sfcWind 
    #   RH       = relative humidity, % = hurs

    #  press   = surface pressure, Pascals = ps
     """
    #Get land and ice masks

    temp_variable = "tasmax"
    f_tmax =cdms.open(fname)
    Tmax = f_ta(temp_variable)
    f_ta.close()
    
    f_tmin =cdms.open(fname.replace(temp_variable,"tmin"))
    Tmin = f_tmin("tmin")
    f_hfls.close()
    
    #Mask the land
    
#     fland = cdms.open(cmip5.landfrac(fname))
#     land = fland("sftlf")
#     fland.close()
#     try:
#         fglac = cdms.open(cmip5.glacierfrac(fname))
    
#         glacier=fglac("sftgif")
#         fglac.close()
#     except:
#         glacier = MV.zeros(land.shape)
#     #mask ocean and ice sheets
#     totmask = np.logical_or(land==0,glacier==100.)
#     nt = len(Ta.getTime())

#    totmask =np.repeat(totmask.asma()[np.newaxis],nt,axis=0)

    winddirec=fname.split(temp_variable+".")[0].replace(temp_variable,"sfcWind")
    fname_wind = fname.replace(temp_variable,"sfcWind")
   
    
    if not (fname_wind in glob.glob(winddirec+"*")):
        #if no surface wind is available, use NCEP reanalyis
        f_wind = cdms.open("wspd.mon.ltm.nc")
        wspd = f_wind("wspd")
        ny=nt/12
        u=np.repeat(wspd,ny,axis=0) #Repeat monthly climatologies
        u.setAxisList([Ta.getTime()]+wspd.getAxisList()[1:])
        u.id="sfcWind"
        f_wind.close()
    else:
        f_wind = cdms.open(fname.replace(temp_variable,"sfcWind"))
        u = f_wind("sfcWind")
        
        f_wind.close()

    
    
    f_hfls =cdms.open(fname.replace(temp_variable,"hfls"))
    hfls = f_hfls("hfls")
    f_hfls.close()
    # Some models put sfcWind on a different grid to calculate surface fluxes
    if u.shape != hfls.shape:
        u = u.regrid(hfls.getGrid(),regridTool='regrid2')
    
    f_hfss =cdms.open(fname.replace(temp_variable,"hfss"))
    hfss = f_hfss("hfss")
    f_hfss.close()

    Rnet = hfss + hfls
    
        

    

    f_hurs =cdms.open(fname.replace(temp_variable,"hurs"))
    RH = f_hurs("hurs")
    f_huss.close()

    f_ps =cdms.open(fname.replace(temp_variable,"ps"))
    press = f_ps("ps")
    f_ps.close()

    
    
    PET,VPD= penmont_vpd_SH(Tmax,Tmin,Rnet,RH,press,u)

    #PET = MV.masked_where(totmask,PET)
    PET.setAxisList(Ta.getAxisList())
    PET.id = "pet"

    #VPD = MV.masked_where(totmask,VPD)
    VPD.setAxisList(Ta.getAxisList())
    VPD.id = "vpd"

   

    
    
    return PET,VPD

In [16]:
def allPET(model,experiment,rip):
    
    fnames=cmip6.get_filenames(model,"tasmax",experiment,rip)
    L=len(fnames)
    i=0
    PET,VPD=PET_from_cmip(fnames[i])
    for i in range(L)[1:]:
        PETi,VPDi,RHi=PET_from_cmip(fnames[i])
        PET=MV.concatenate((PET,PETi))
        VPD=MV.concatenate((VPD,VPDi))
        
    tax=get_tax_from_files(fnames)
    PET.setAxis(0,tax)
    PET.id="pet"
    VPD.setAxis(0,tax)
    VPD.id="vpd"
    
    
    return PET,VPD

In [20]:
model="CESM2"
experiment="historical"
rip="r1i1p1f1"
cmip6.get_filenames(model,"tasmin",experiment,rip)

[]

In [6]:
#Write the potential evapotranspiration (calculated with Penman-Monteith), VPD, and RH
def writePET(curr_mod,experiment_id,member_id):
    base_write_dir= '/home/kdm2144/DROUGHT/PROCESSED/'
    for curr_var in ["pet","vpd"]:
        os.makedirs(os.path.join(base_write_dir, curr_var, curr_mod),exist_ok=True)
    fnames=cmip6.get_filenames(curr_mod,"tas",experiment_id,member_id)
    for fname in fnames:
       
        d={}
        pet,vpd=PET_from_cmip(fname)
        d["pet"]=pet
        d["vpd"]=vpd
        
        for curr_var in ["pet","vpd"]:
            write_dir = base_write_dir+curr_var+"/"+curr_mod+"/"
            year=fname.split(".")[-2]
            writename = curr_var+"."+experiment_id+"."+curr_mod+"."+member_id+"."+year.zfill(4)+".nc"
            fw=cdms.open(write_dir+writename,"w")
            fw.write(d[curr_var])
            fw.close()
        
    
    

# MOVE TO CMIP6HELPER

In [ ]:
mary=pd.read_csv("/home/kdm2144/mary_cmip6.csv")
def cmip6_models_with_all_variables(variables,experiment):
    models=np.unique(mary[(mary.experiment_id==experiment) & (mary.variable_id==variables[0])].source_id)
    for variable in variables[1:]:
        models2=np.unique(mary[(mary.experiment_id==experiment) & (mary.variable_id==variable)].source_id)
        models=np.intersect1d(models,models2)
    return models

def subset_CMIP6_models(variables,experiments):
    models=cmip6_models_with_all_variables(variables,experiments[0])
    for experiment in experiments:
        models2=cmip6_models_with_all_variables(variables,experiment)
        models=np.intersect1d(models,models2)
    return models




In [15]:
def get_filenames(model,variable,experiment,rip):
    
    return sorted(glob.glob("/home/kdm2144/DROUGHT/DOWNLOADED_RAW/"+"/"+variable+"/"+model+"/*.historical.*."+rip+".*"))

In [16]:
def get_rips(model,variable,experiment):
    rips=np.unique([x.split(".")[-3] for x in glob.glob("/home/kdm2144/DROUGHT/DOWNLOADED_RAW/"+"/"+variable+"/"+model+"/*"+experiment+"*")])
    return rips


In [19]:
def get_common_rips(model,variables,experiment):
    rips=get_rips(model,variables[0],experiment)
    for variable in variables[1:]:
        rips2=get_rips(model,variable,experiment)
        rips=np.intersect1d(rips,rips2)
    return rips

In [17]:
def get_tax(experiment,L=None):
    if experiment.find("hist")==0:
        if L is None:
            L=165
        tax=cdms.createAxis(np.arange(L*12))
        tax.designateTime()
        tax.id="time"
        tax.units="months since 1850-1-1"
    elif experiment.find("ssp")==0:
        if L is None:
            L=86
        tax=cdms.createAxis(np.arange(L*12))
        tax.designateTime()
        tax.id="time"
        tax.units="months since 1850-1-1"
    elif experiment.find("piControl")==0:
        if L is None:
            L=500
        tax=cdms.createAxis(np.arange(L*12))
        tax.designateTime()
        tax.id="time"
        tax.units="months since 0001-1-1"
    cdutil.setAxisTimeBoundsMonthly(tax)
    return tax

In [5]:
def get_tax_from_files(fnames):

    minyear=int(fnames[0].split(".")[-2])
    maxyear=int(fnames[-1].split(".")[-2])
    L=maxyear-minyear+1
    tax=cdms.createAxis(np.arange(L*12))
    tax.designateTime()
    tax.id="time"
    tax.units="months since "+str(minyear)+"-1-1"
    cdutil.setAxisTimeBoundsMonthly(tax)
    return tax

In [14]:
pet_variables=["hfls","hfss","sfcWind","huss","ps","tas"]
experiments=["historical","ssp126","ssp585","ssp370","ssp245","piControl"]

models=subset_CMIP6_models(pet_variables,experiments)

In [21]:
write_all_pet_files=False
if write_all_pet_files:
    for experiment in experiments:
        for model in models:
            rips=get_common_rips(model,pet_variables,experiment)
            for rip in rips:
                try:
                    writePET(model,experiment,rip)
                except:
                    print("Problem with "+model+"."+experiment+"."+rip)

array(['r1i1p1f1', 'r2i1p1f1'], dtype='<U8')